In [1]:
import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp

np.random.seed(12)
samples = 500
probability = 0.2

index = ['name', 'cost', 'size', 'handling', 'positioning', 'joining']
names = ['robot' + str(i) for i in range(samples)]
costs = np.random.randint(1000, 10000, size=(samples)).tolist()
handling = np.random.binomial(1, probability, samples)
positioning = np.random.binomial(1, probability, samples)
joining = np.random.binomial(1, probability, samples)
sizes = np.round(np.random.rand((samples)) * 10, 2)

df = pd.DataFrame({'name': names, 'cost': costs, 'size': sizes, 'handling': handling, 'positioning': positioning, 'joining': joining})
df.head(10)

,name,cost,size,handling,positioning,joining
0,robot0,6787,9.98,0,0,0
1,robot1,4325,0.42,1,0,1
2,robot2,8409,1.20,1,1,0
3,robot3,4714,8.71,0,0,0
4,robot4,1278,0.68,1,0,0
5,robot5,9241,8.38,0,0,1
6,robot6,4725,7.91,0,1,1
7,robot7,5569,0.43,0,0,1
8,robot8,9610,6.49,0,0,0
9,robot9,9651,4.94,1,0,0


In [2]:
constraints = {'cost': 20000, 'size': 10.0, 'handling': 8, 'positioning': 5, 'joining': 3}

## Create Variables

In [3]:
solver = pywraplp.Solver('simple_lp_programm', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
solver.Clear()

robot = [[]] * len(df)
# create variables
for i in range(len(df)):
    robot[i] = solver.IntVar(0, solver.infinity(), df['name'][i])
    
print('Number of variables:', solver.NumVariables())

Number of variables: 500


## Fitness Function
\begin{equation}
f = \sum_{r \in \mathcal{R}} \frac{c_r}{C_{max}} + \frac{s_r}{S_{max}} \\
\text{with:} \\
\text{f = fitness} \\
\text{c = cost} \\
\text{s = size}
\end{equation}

In [4]:
def fitness(robot):
    l = lambda x: df.loc[df.name==str(robot), x].values[0]
    f = l('cost') / constraints['cost'] + l('size') / constraints['size']
    return f

## Create Objective Function

In [5]:
# create objective function
solver.Minimize(solver.Sum( [fitness(robot[i]) for i in range(len(robot))] ))

In [6]:
ll = lambda s: [robot[i] * df[s][i] for i in range(len(df))]

## Create Constraints

In [7]:
# create constraints
solver.Add(0 <= solver.Sum(ll('cost')) <= constraints['cost'])
solver.Add(0 <= solver.Sum(ll('size')) <= constraints['size'])
solver.Add(constraints['handling'] <= solver.Sum(ll('handling')))
solver.Add(constraints['positioning'] <= solver.Sum(ll('positioning')))
solver.Add(constraints['joining'] <= solver.Sum(ll('joining')))

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x000001DF3176D4B0> >

In [8]:
result_status = solver.Solve()
# The problem has an optimal solution.
assert result_status == pywraplp.Solver.OPTIMAL

# The solution looks legit (when using solvers others than
# GLOP_LINEAR_PROGRAMMING, verifying the solution is highly recommended!).
assert solver.VerifySolution(1e-4, True)

print('Solution:')
print('Number of robots:', sum([robot[i].solution_value() for i in range(len(robot))]))

cost = 0
size = 0
handling = 0
positioning = 0
joining = 0
robots = []
for i in range(len(df)):
    if robot[i].solution_value() > 0: 
        c = df.loc[df.name==str(robot[i]), 'cost'].values[0]
        s = df.loc[df.name==str(robot[i]), 'size'].values[0]
        handling += df.loc[df.name==str(robot[i]), 'handling'].values[0] * robot[i].solution_value()
        positioning += df.loc[df.name==str(robot[i]), 'positioning'].values[0] * robot[i].solution_value()
        joining += df.loc[df.name==str(robot[i]), 'joining'].values[0] * robot[i].solution_value()
        robots.append(str(robot[i]))
        print('{:<10} | num: {:<5} | cost: {:<5} € | size: {:<5} m2'.format(str(robot[i]), robot[i].solution_value(), c, s))
        cost += c
        size += s

print('Cost =', cost, '€')
print('Size =', round(size, 3), 'm2')
print('Handling =', handling, '/', constraints['handling'])
print('Positioning =', positioning, '/', constraints['positioning'])
print('Joining =', joining, '/', constraints['joining'])

Solution:
Number of robots: 14.0
robot4     | num: 7.0   | cost: 1278  € | size: 0.68  m2
robot61    | num: 2.0   | cost: 1160  € | size: 0.16  m2
robot190   | num: 1.0   | cost: 2816  € | size: 1.75  m2
robot470   | num: 1.0   | cost: 1638  € | size: 0.41  m2
robot477   | num: 3.0   | cost: 1034  € | size: 0.84  m2
Cost = 7926 €
Size = 3.84 m2
Handling = 8.0 / 8
Positioning = 5.0 / 5
Joining = 3.0 / 3


**Objective here is to minimize the fitness function**

In [9]:
df.loc[df.name.isin(robots), :]

,name,cost,size,handling,positioning,joining
4,robot4,1278,0.68,1,0,0
61,robot61,1160,0.16,0,0,1
190,robot190,2816,1.75,1,1,1
470,robot470,1638,0.41,0,1,0
477,robot477,1034,0.84,0,1,0
